In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os, re, glob
from astropy.io import fits
from functions import read_data, cropped, createIm, normalization, saveIm, binning, oscillation
sns.set()
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
cwd = os.getcwd()
parent_cwd = os.path.abspath(os.path.join(cwd, os.pardir))
display(cwd, parent_cwd)

'/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook'

'/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018'

In [2]:
def reader(path, crop_param=[0, 0, 510, 510]):

    with fits.open(path, memmap=False) as f:
        return f[0].data[crop_param[0]:crop_param[2], crop_param[1]:
                         crop_param[3]]


def stackChunkCreator(df_jump_end, crop_param=[0, 0, 510, 510]):

    stack = []
    print('reading...')
    for name, column in df_jump_end.iteritems():
        stack.append(column.apply(reader, crop_param=crop_param).sum())
    return stack


def runningChunk(df_G0_smp,
                 df_G0_OB,
                 displayViewPlot=False,
                 lenstep=100,
                 overlapP=0.05,
                 crop_param=[0, 0, 510, 510],
                 norm_param=[484, 5, 24, 500],
                 bin_fac=3,
                 safeTapy=False,
                 saveFolder='results'):

    overlap = int(lenstep * overlapP)
    print('chunksize: ', lenstep, '\toverlap: ', overlap, '\tDelta: ',
          lenstep - overlap)
    counter = 0

    if len(df_G0_smp) == len(df_G0_OB):
        counter0 = 0
        for jump, end in zip(
                range(0, len(df_G0_smp), lenstep - overlap),
                range(lenstep,
                      len(df_G0_smp) + lenstep, lenstep - overlap)):
            if end > len(df_G0_smp):
                end = len(df_G0_smp)
            nameStep = saveFolder + '_' + str(counter0).zfill(5)
            counter0 += 1
            print('index start:', jump, ' end: ', end,'nameStep: ',nameStep)
            df_G0_smp_jump_end = df_G0_smp[jump:end]
            df_G0_OB_jump_end = df_G0_OB[jump:end]
            if displayViewPlot:
                display('df_G0_smp_jump_end', df_G0_smp_jump_end.head(1),
                        df_G0_smp_jump_end.tail(1))
                print('len df_G0_smp_jump_end: ', len(df_G0_smp_jump_end))
                display('df_G0_OB_jump_end', df_G0_OB_jump_end.head(1),
                        df_G0_OB_jump_end.tail(1))
            im_stack = stackChunkCreator(
                df_G0_smp_jump_end, crop_param=crop_param)
            OB_stack = stackChunkCreator(
                df_G0_OB_jump_end, crop_param=crop_param)
           
            numberPeriods = np.shape(im_stack)[0] / np.shape(OB_stack)[0]
            print('numberPeriods:\t',numberPeriods)

            im_stack = im_stack
            OB_stack = OB_stack[:np.shape(im_stack)[0]]
            if len(im_stack) is not len(OB_stack):
                print('Error \t len(im_stack) is not len(OB_stack)')
            norm_param = norm_param
            im, ob = normalization(im_stack, OB_stack, *norm_param)
            im, ob = binning(im, ob, bin_fac)
            ti, dpci, dfi, vis_map = createIm(im, ob, numberPeriods)

            if safeTapy:
                saveIm(
                    ti,
                    dpci,
                    dfi,
                    vis_map,
                    name=nameStep,
                    folder=os.path.join('results', saveFolder),
                    overWrite=True)

        print(50 * '+ ')
    else:
        print('len(df_G0_smp):\t', len(df_G0_smp), 'len(df_G0_OB):\t',
              len(df_G0_OB))


def dfCreator(path, displayView=False):
    if os.path.exists(path):
        print(os.path.exists(path), path)
        df = pd.DataFrame(sorted(os.listdir(path)), columns=['FileName'])
        df['G0pos'] = df['FileName'].apply(lambda x: '0.' + re.search(
            '_0.(.*)', x).group(1).replace('_Corrected', ''))
        df_G0 = pd.DataFrame(columns=df['FileName'].values.tolist())

        for i in df_G0.columns.values.tolist():
            df_G0[i] = sorted(
                glob.glob((os.path.join(path, i, '*_Corrected_0*.fits'))))

        print('G0 steps', len(df), 'images for each G0', len(df_G0))

        if displayView:
            display(df.head(1), df.tail(1), df_G0.head(1), df_G0.tail(1))

        return df_G0

    else:
        return pd.DataFrame()


def tapyRunner(df_smp_path,
               df_Ob_path,
               lenstep=150,
               overlapP=0.05,
               crop_param=[0, 0, 510, 510],
               norm_param=[484, 5, 24, 500],
               bin_fac=3,
               displayView=True,
               displayViewPlot=False,
               safeTapy=True,
               saveFolder='results'):

    df_smp = dfCreator(df_smp_path)
    df_Ob = dfCreator(df_Ob_path)
    runningChunk(
        df_smp,
        df_Ob,
        lenstep=lenstep,
        crop_param=crop_param,
        norm_param=norm_param,
        bin_fac=bin_fac,
        safeTapy=safeTapy,
        saveFolder=saveFolder+'_pbin'+str(bin_fac)+'_tbin'+str(lenstep).zfill(4))

# Open Beam summed

In [3]:
obSum_folder = os.path.join(parent_cwd, 'Data', 'Corrected', 'OB_sum_corrected')

# sample smp_carb_30min

In [4]:
smp_carb_30min = 'smp_carb_30min'
smp_carb_30min_folder = os.path.join(parent_cwd, 'Data', 'Corrected', 'smp',
                                     smp_carb_30min)
tapyRunner(
    smp_carb_30min_folder,
    obSum_folder,
    lenstep=120,
    overlapP=0.05,
    crop_param=[0, 0, 510, 510],
    bin_fac=3,
    norm_param=[484, 280, 24, 225],saveFolder=smp_carb_30min)

tapyRunner(
    smp_carb_30min_folder,
    obSum_folder,
    lenstep=25,
    overlapP=0.1,
    crop_param=[0, 0, 510, 510],
    bin_fac=3,
    norm_param=[484, 280, 24, 225],saveFolder=smp_carb_30min)

# smp_tung_pow310_60min

In [5]:
smp_tung_pow310_60min = 'smp_tung_pow310_60min'
smp_tung_pow310_60min_folder = os.path.join(parent_cwd, 'Data', 'Corrected', 'smp',
                                     smp_tung_pow310_60min)

norm_param=[484, 5, 24, 235]
crop_param=[0, 0, 510, 510]
bin_fac=3

tapyRunner(
    smp_tung_pow310_60min_folder,
    obSum_folder,
    lenstep=120,
    overlapP=0.05,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,saveFolder=smp_tung_pow310_60min)

tapyRunner(
    smp_tung_pow310_60min_folder,
    obSum_folder,
    lenstep=25,
    overlapP=0.1,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,saveFolder=smp_tung_pow310_60min)

True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/smp/smp_tung_pow310_60min
G0 steps 15 images for each G0 2980
True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/OB_sum_corrected
G0 steps 15 images for each G0 2980
chunksize:  120 	overlap:  6 	Delta:  114
index start: 0  end:  120 nameStep:  smp_tung_pow310_60min_pbin3_tbin0120_00000
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0120
index start: 114  end:  234 nameStep:  smp_tung_pow310_60min_pbin3_tbin0120_00001
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0120
index start: 228  end:  348 nameStep:  smp_tung_pow310_60min_pbin3_tbin0120_00002
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0120
index start: 342  end:  462 nameStep:  smp_tung_pow310_60min_pbin3_tbin0120_00003
reading...
reading...
numberPeriods:	 1.0
files saved 

reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 312  end:  337 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00013
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 336  end:  361 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00014
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 360  end:  385 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00015
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 384  end:  409 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00016
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 408  end:  433 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00017
reading...
reading...
numberPeriods:	 1.0
file

reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 1344  end:  1369 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00056
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 1368  end:  1393 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00057
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 1392  end:  1417 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00058
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 1416  end:  1441 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00059
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 1440  end:  1465 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00060
reading...
reading...
numberPeriods:

numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2352  end:  2377 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00098
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2376  end:  2401 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00099
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2400  end:  2425 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00100
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2424  end:  2449 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00101
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2448  end:  2473 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00102
reading...
reading...
numberPeriods:	 1.0
files

/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook/functions.py:437: RuntimeWarning: invalid value encountered in true_divide
  phi = np.reshape(np.arctan((np.asarray(absPhase)/np.asarray(absoluteAmpl))),[shapeStack[1],shapeStack[2]])
/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook/functions.py:539: RuntimeWarning: invalid value encountered in true_divide
  DFI = np.divide(np.divide(imParam[1],imParam[0]),np.divide(obParam[1],obParam[0]))


index start: 2736  end:  2761 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00114
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025


/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook/functions.py:536: RuntimeWarning: divide by zero encountered in true_divide
  TI = np.divide(imParam[0],obParam[0])
/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook/functions.py:536: RuntimeWarning: invalid value encountered in true_divide
  TI = np.divide(imParam[0],obParam[0])
/Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/notebook/functions.py:540: RuntimeWarning: invalid value encountered in true_divide
  VIS_map = np.divide(obParam[1],obParam[0])


index start: 2760  end:  2785 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00115
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2784  end:  2809 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00116
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2808  end:  2833 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00117
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2832  end:  2857 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00118
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2856  end:  2881 nameStep:  smp_tung_pow310_60min_pbin3_tbin0025_00119
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_tung_pow310_60min_pbin3_tbin0025
index start: 2880  end:  

# smp_weld_40min

In [6]:
smp_weld_40min = 'smp_weld_40min'
smp_weld_40min_folder = os.path.join(parent_cwd, 'Data', 'Corrected', 'smp',
                                     smp_weld_40min)

norm_param = [484, 5, 24, 500]
crop_param = [0, 0, 510, 510]
bin_fac = 3

tapyRunner(
    smp_weld_40min_folder,
    obSum_folder,
    lenstep=120,
    overlapP=0.05,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,saveFolder=smp_weld_40min)

tapyRunner(
    smp_weld_40min_folder,
    obSum_folder,
    lenstep=25,
    overlapP=0.1,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,
    saveFolder=smp_weld_40min)

True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/smp/smp_weld_40min
G0 steps 15 images for each G0 2980
True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/OB_sum_corrected
G0 steps 15 images for each G0 2980
chunksize:  120 	overlap:  6 	Delta:  114
index start: 0  end:  120 nameStep:  smp_weld_40min_pbin3_tbin0120_00000
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0120
index start: 114  end:  234 nameStep:  smp_weld_40min_pbin3_tbin0120_00001
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0120
index start: 228  end:  348 nameStep:  smp_weld_40min_pbin3_tbin0120_00002
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0120
index start: 342  end:  462 nameStep:  smp_weld_40min_pbin3_tbin0120_00003
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0120
index 

reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 384  end:  409 nameStep:  smp_weld_40min_pbin3_tbin0025_00016
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 408  end:  433 nameStep:  smp_weld_40min_pbin3_tbin0025_00017
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 432  end:  457 nameStep:  smp_weld_40min_pbin3_tbin0025_00018
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 456  end:  481 nameStep:  smp_weld_40min_pbin3_tbin0025_00019
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 480  end:  505 nameStep:  smp_weld_40min_pbin3_tbin0025_00020
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start:

reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 1488  end:  1513 nameStep:  smp_weld_40min_pbin3_tbin0025_00062
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 1512  end:  1537 nameStep:  smp_weld_40min_pbin3_tbin0025_00063
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 1536  end:  1561 nameStep:  smp_weld_40min_pbin3_tbin0025_00064
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 1560  end:  1585 nameStep:  smp_weld_40min_pbin3_tbin0025_00065
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 1584  end:  1609 nameStep:  smp_weld_40min_pbin3_tbin0025_00066
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
in

index start: 2568  end:  2593 nameStep:  smp_weld_40min_pbin3_tbin0025_00107
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 2592  end:  2617 nameStep:  smp_weld_40min_pbin3_tbin0025_00108
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 2616  end:  2641 nameStep:  smp_weld_40min_pbin3_tbin0025_00109
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 2640  end:  2665 nameStep:  smp_weld_40min_pbin3_tbin0025_00110
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 2664  end:  2689 nameStep:  smp_weld_40min_pbin3_tbin0025_00111
reading...
reading...
numberPeriods:	 1.0
files saved in folder:  results/smp_weld_40min_pbin3_tbin0025
index start: 2688  end:  2713 nameStep:  smp_weld_40min_pbin3_tbin0025_00112
reading...
reading

# smp3_ebw_Cu_SS_1bottom_3top

In [ ]:
smp3_ebw_Cu_SS_1bottom_3top = 'smp3_ebw_Cu_SS_1bottom_3top'
smp3_ebw_Cu_SS_1bottom_3top_folder = os.path.join(
    parent_cwd, 'Data', 'Corrected', 'smp', smp3_ebw_Cu_SS_1bottom_3top)

norm_param = [486, 5, 23, 500]
crop_param = [0, 0, 510, 510]
bin_fac = 3

tapyRunner(
    smp3_ebw_Cu_SS_1bottom_3top_folder,
    obSum_folder,
    lenstep=120,
    overlapP=0.05,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,
    saveFolder=smp3_ebw_Cu_SS_1bottom_3top)

tapyRunner(
    smp3_ebw_Cu_SS_1bottom_3top_folder,
    obSum_folder,
    lenstep=25,
    overlapP=0.1,
    crop_param=crop_param,
    bin_fac=bin_fac,
    norm_param=norm_param,
    saveFolder=smp3_ebw_Cu_SS_1bottom_3top)

True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/smp/smp3_ebw_Cu_SS_1bottom_3top
G0 steps 9 images for each G0 2980
True /Volumes/nGI_IMAT_18/IMAT_nGI_optimized_2018/Data/Corrected/OB_sum_corrected
G0 steps 15 images for each G0 2980
chunksize:  120 	overlap:  6 	Delta:  114
index start: 0  end:  120 nameStep:  smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120_00000
reading...
reading...
numberPeriods:	 0.6
files saved in folder:  results/smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120
index start: 114  end:  234 nameStep:  smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120_00001
reading...
reading...
numberPeriods:	 0.6
files saved in folder:  results/smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120
index start: 228  end:  348 nameStep:  smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120_00002
reading...
reading...
numberPeriods:	 0.6
files saved in folder:  results/smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120
index start: 342  end:  462 nameStep:  smp3_ebw_Cu_SS_1bottom_3top_pbin3_tbin0120_00003
reading